# L2: Information Extraction

In this lab you will implement and evaluate a simple system for information extraction. The task of the system is to read sentences and extract pairs of the form $(x, y)$, where $x$&nbsp;is a string denoting a person, $y$&nbsp;is a string denoting an organisation, and $x$ is the &lsquo;leader&rsquo; of&nbsp;$y$. Consider the following example sentence:

<blockquote>
Mr. Obama also selected Lisa Jackson to head the Environmental Protection Agency.
</blockquote>

From this sentence the system should extract the pair
```
("Lisa Jackson", "Environmental Protection Agency")
```

The system will have to solve the following sub-tasks:
* entity extraction &ndash; identifying mentions of person and organisation entities in text
* relation extraction &ndash; identifying instances of the &lsquo;is-leader-of&rsquo; relation

The data set for the lab consists of 62,010&nbsp;sentences from the [Groningen Meaning Bank](http://gmb.let.rug.nl) (release 2.2.0), an open corpus of English. To analyse the sentences you will use [spaCy](https://spacy.io/).

## Getting started

The first cell imports the Python module required for this lab.

In [1]:
import tm2

The next cell imports spaCy and loads its English language model.

In [2]:
import spacy

nlp = spacy.load('en')

## Data and gold standard

The data is contained in the following file:

In [3]:
data_file = "/home/TDDE16/labs/l2/data/gmb.txt"

The `tm2` module defines a function `read_data` that returns an iterator over the lines in a file. You should use this function to read the data for this lab. Use the optional argument `n` to restrict the iteration to the first few lines of the file. Here is an example:

In [4]:
for sentence in tm2.read_data(data_file, n=3):
    print(sentence)

Masked assailants with grenades and automatic weapons attacked a wedding party in southeastern Turkey, killing 45 people and wounding at least six others.
Turkish officials said the attack occurred Monday in the village of Bilge about 600 kilometers from Ankara.
The wounded were taken to the hospital in the nearby city of Mardin.


In addition to the raw data, we also provide you with a gold standard of entity pairs that your system should be able to extract. The following code loads these pairs from the file `gold.txt` and adds them to the set `gold`. Each pair is augmented with the identifier of the sentence (line number in the data file) which it was extracted from. Note that the sentence (line) numbering starts at index&nbsp;0.

In [5]:
gold_file = "/home/TDDE16/labs/l2/data/gold.txt"

gold = set()
with open(gold_file) as fp:
    for line in fp:
        columns = line.rstrip().split('\t')
        gold.add((int(columns[0]), columns[1], columns[2]))

The following code prints the 10&nbsp;first pairs from the gold standard:

In [6]:
for i, person, org in sorted(gold)[:10]:
    print("{}\t{}\t{}".format(i, person, org))

802	Ali Zardari	Pakistan People 's Party
2297	Abdul Aziz al-Hakim	Supreme Council
4823	Slavkov	Bulgarian National Olympic Committee
7902	Mr. Hakim	Supreme Council
8206	J. Patrick Boyle	American Meat Institute
8633	Ali Rodriguez	Petroleos de Venezuela
9004	Foreign Minister Joschka Fischer	Green Party
11021	Khalaf	al-Qaida
11259	Joseph Domenech	U.N. 's Food and Agricultural Organization
13043	David Petraeus	U.S. Central Command


You should take a moment to have a look at the data file and see these pairs in the context of the sentence they were extracted from.

In [144]:
sentences = list()
for sentence in tm2.read_data(data_file, n=682):
    sentences.append(sentence)

In [145]:
sentences[680]

'Araujo said Mr. Chavez and the leftist observers came to criticize the government and defend the guerrillas.'

100


TypeError: Argument 'string' has incorrect type (expected str, got set)

## Entity extraction

To implement the entity extraction part of your system, you do not need to do much, as you can use the full natural language processing power built into spaCy. The following code extracts the entities from the first 5&nbsp;sentences of the data:

In [167]:
for i, doc in enumerate(nlp.pipe(tm2.read_data(data_file, n=370))):
    if i == 361:
        for ent in doc.ents:
            print("{}\t{}\t{}\t{}".format(ent.text, ent.start, ent.end, ent.label_))

BP	1	2	ORG
Robert Dudley	3	5	PERSON
Russia	10	11	GPE
next week	11	13	DATE
first	15	16	ORDINAL
2008	22	23	DATE
BP	28	29	ORG
Russian	31	32	NORP
TNK	33	34	ORG
BP	35	36	ORG


Read the [section about named entities](https://spacy.io/usage/linguistic-features#section-named-entities) from spaCy&rsquo;s documentation to get some background on this.

## Problem 1: Extract relevant pairs

The first problem that you will have to solve is to identify pairs of entities that are in the &lsquo;is-leader-of&rsquo; relation, as in the example above. There are many ways to do this, but for this lab it suffices to implement the strategy outlined in the section on [Relation Extraction](http://www.nltk.org/book/ch07.html#relation-extraction) in the book by Bird, Klein, and Loper (2009):

* look for all triples of the form $(x, \alpha, y)$ where $x$ and $y$ denote named entities of type *person* and *organisation*, respectively, and $\alpha$ is the intervening text
* write a regular expression to match just those instances of $\alpha$ that express the &lsquo;is-leader-of&rsquo; relation

You can restrict your attention to adjacent pairs of entities &ndash; that is, cases where $x$ precedes $y$ and $\alpha$ does not contain other named entities.


Write a function `extract` that takes an analysed sentence (represented as a spaCy [`Doc`](https://spacy.io/api/doc) object) and yields pairs $(x, y)$ of strings representing entity mentions predicted to be in the &lsquo;is-leader-of&rsquo; relation.

In [142]:
import re
def extract(doc):
    synonyms = nlp(u'leader chief director officer boss president king')
    j = -1
    for token1 in doc:
        j = j+1 # position of the token in the word
        for token2 in synonyms:
            if token1.text == token2.text: 
                    print(i,j)
                    x = list()
                    y = list()
                    between_start = 0
                    between_end = 0
                    #print(doc.ents)
                    for ent in doc.ents:
                        between_text = str()
                        if ent.label_ == "PERSON" and ent.end < j:
                            between_start = ent.end+1
                            x.append(ent.text)
                        if (between_start > 0 and ent.label_ == "ORG" and ent.start > j):
                            between_end = ent.start -1
                            y.append(ent.text)
                            between_text = doc[between_start:between_end]
                            #result = result + ((i,x[0],between_text,y[0]),)
                            result = result + ((i,x[0],y[0]),)
    return(result)
    # TODO: Replace the following line with your own code
    #return tm2.extract(doc)

In [163]:
result = ()
for i, doc in enumerate(nlp.pipe(tm2.read_data(data_file, n=1000))):
    if i == 361:
        x = list()
        y = list()
        between_start = 0
        between_end = 0
        print(doc.ents)
        for ent in doc.ents:
            between_text = str()
            if ent.label_ == "PERSON" :
                between_start = ent.end+1
                x.append(ent.text)
            if (between_start > 0 and ent.label_ == "ORG"):
                between_end = ent.start -1
                y.append(ent.text)
                between_text = doc[between_start:between_end]
                result = result + ((i,x[0],between_text,y[0]),)
            

(BP, Robert Dudley, Russia, next week, first, 2008, BP, Russian, TNK, BP)


In [194]:
result = ()
for i, doc in enumerate(nlp.pipe(tm2.read_data(data_file, n=1000))):
    #synonyms = ["leader","chief","director","officer","boss","president","king"]
    synonyms = nlp(u'leader chief director officer boss president king')
    j = -1
    for token1 in doc:
        j = j+1 # position of the token in the word
        for token2 in synonyms:
            if token1.text == token2.text: 
                    print(i,j)
                    x = list()
                    y = list()
                    between_start = 0
                    between_end = 0
                    #print(doc.ents)
                    for ent in doc.ents:
                        between_text = str()
                        if ent.label_ == "PERSON" and ent.end < j:
                            between_start = ent.end+1
                            x.append(ent.text)
                        if (between_start > 0 and ent.label_ == "ORG" and ent.start > j):
                            between_end = ent.start -1
                            y.append(ent.text)
                            between_text = doc[between_start:between_end]
                            result = result + ((i,x[0],between_text,y[0]),)
#     lower_doc = [word.word for word in doc]
#     print("lower_doc ",lower_doc)
#     for word in synonyms:
#         a = r'(.*)' + word + '(.*?) .*'
#         print(a)
#         matchOBj = re.match(a,lower_doc)
#         print(matchOBj)
# print(result)
# sentences[361]


48 1
(the International Atomic Energy Agency, Mohamed ElBaradei, the Nobel Peace Prize)
63 4
(Mohamed ElBaradei, Tuesday, Arab)
124 5
(Jose Manuel Barroso, Monday)
141 9
(Palestinian, Israeli, Moshe Yaalon, the Gaza Strip, Palestinian)
149 2
(Iranian, Bush, Iran, U.S., one)
149 19
(Iranian, Bush, Iran, U.S., one)
153 3
(EU, Javier Solana, Tehran, later this week)
166 1
(the Portuguese Football Federation, Portugal, Asian, last weekend's)
176 6
(Burma, Independence Day, this week)
182 13
()
207 21
(8:20 am, Rugova, European Union, Javier Solana)
210 1
()
218 18
(Venezuelans,)
220 12
()
283 3
(Michael Green, Asian, the U.S. National Security Council, Bush, Pyongyang)
291 3
(Hamas, Islamic, Israel, the end of this year)
306 1
(Palestinian, Mahmoud Abbas, first, Israel, Jewish, the Gaza Strip, earlier this month)
310 14
(Pakistan, Taleban, U.S, Afghanistan)
362 4
(Dudley, TNK, BP, Moscow, BP, Tony Hayward)
362 18
(Dudley, TNK, BP, Moscow, BP, Tony Hayward)
376 4
(Friday, Senate, January 9,

In [155]:
import re
final_result = ()
for item in result:
    a = re.match(r'(.*) of (.*?) .*',str(item[2]))
    b = re.match(r'(.*) from (.*?) .*',str(item[2]))
    #print(item[2])
    #print("matched:", a, b)
    if a is not None or b is not None:
        final_result = final_result + ((item[1],item[3]),)
        
    
    

In [195]:
sentences[48]
result

((283,
  'Michael Green',
  director for Asian affairs,
  'the U.S. National Security Council'),
 (736, 'Viktor Yanukovych', the leader of the opposition, 'Russian Party'),
 (802, 'Asif Ali Zardari', leader, "the Pakistan People's Party"))

The following cell shows how your function is supposed to be used. The code prints out the extracted pairs for the first 1,000&nbsp;sentences in the data. It additionally numbers each pair with the sentence identifier.

In [143]:
for i, doc in enumerate(nlp.pipe(tm2.read_data(data_file, n=1000))):
    for person, org in extract(doc):
        print("{}\t{}\t{}".format(i, person, org))

(9, 'Kofi Annan', Haiti is at a critical juncture, as the country prepares for its first set of elections since the ouster of, 'Jean-Bertrand Aristide')
9	Kofi Annan	Jean-Bertrand Aristide
(42, 'Ahmad', who are members of a pro-Syrian Islamic militant group, 'Al-Ahbash')
42	Ahmad	Al-Ahbash
(638, 'Hassan', Iraqi operations of the international, 'CARE')
638	Hassan	CARE
(681, 'Chavez', the observers to oversee the release of three hostages held, 'the Revolutionary Armed Forces of Colombia')
(681, 'Chavez', the observers to oversee the release of three hostages held by the Revolutionary Armed Forces of Colombia, 'the Revolutionary Armed Forces of Colombia')
681	Chavez	the Revolutionary Armed Forces of Colombia
681	Chavez	the Revolutionary Armed Forces of Colombia
(736, 'Viktor Yanukovych', the leader of the opposition, 'Russian Party')
736	Viktor Yanukovych	Russian Party
(761, 'Andy Roddick', claimed the remaining qualifying spots for the Association of Tennis Professionals (ATP, 'Tomas Be

Once you feel confident that your `extract` function does what it is supposed to do, execute the following cell to extract the entities from the full data set. Note that this will take several minutes (remember that we are processing 62k sentences).

In [ ]:
extracted = set()
for i, doc in enumerate(nlp.pipe(tm2.read_data(data_file))):
    for person, org in extract(doc):
        extracted.add((i, person, org))
    print('\rProcessed {} sentences ...'.format(i+1), end='', flush=True)
print(' done')

After executing the above cell, all extracted id-string-string triples are in the set `extracted`. The code in the next cell will print the first 10&nbsp;triples in this set.

In [ ]:
for i, person, org in sorted(extracted)[:10]:
    print("{}\t{}\t{}".format(i, person, org))

## Problem 2: Evaluate your system

You now have an extractor, but how good is it? Your task now is to write code that computes the precision, recall, and F1 measure of your extractor relative to the gold standard.

In [ ]:
def evaluate(reference, predicted):
    """Print out the precision, recall, and F1 for the id-entity-entity
    triples in the set `predicted`, given the triples in the reference set.
    
    Args:
        reference: The reference set of triples.
        predicted: The set of predicted triples.
    Returns:
        Nothing, but prints out precision, recall, and F1.
    """
    # TODO: Replace the next line with your own code
    tm2.evaluate(reference, predicted)

The next cell shows how your function is intended to be used, as well as the suggested output format.

In [ ]:
evaluate(gold, extracted)

## Problem 3: Entity resolution

Looking at the results of your quantitative evaluation, you will realise that your extractor (probably) does a rather poor job in matching the gold standard. One reason for this is that the NLP preprocessing is not perfect (spaCy was not trained on the annotations in the Groningen Meaning Bank), and that the approach of using regular expressions for relation extraction is rather naive.

Another reason however is that the current version of your system does not include a component for *entity resolution*. To give an example, your system does not realise that the strings `David Petraeus` and `General David Petraeus` refer to the same entity.

While writing a &lsquo;real&rsquo; entity resolver is beyond the scope of this assignment, we ask you to &lsquo;fake&rsquo; such a resolver. More specifically, you should implement a function `normalise` that takes an entity mention (a string) as its input and rewrites it to the form used in the gold standard. While this is &lsquo;cheating&rsquo;, it allows you to assess the performance of a more realistic system, and helps to illustrate that information extraction can be very domain-specific.

The following cell contains skeleton code for the `normalise` function.

In [ ]:
def normalise(text):
    if text == "David Petraeus":
        return "General David Petraeus"
    return text

The next cell shows how `normalise` is intended to be used. Each triple in the set `extracted` is transformed by feeding the two entity mentions into the `normalise` function. The normalised triples are then added to a new set `extracted_normalised`.

In [ ]:
extracted_normalised = set()
for triple in extracted:
    extracted_normalised.add((triple[0], normalise(triple[1]), normalise(triple[2])))

To pass the assignment, you should add enough normalisation rules to `normalise` to achieve a recall of at least 50%.

In [ ]:
evaluate(gold, extracted_normalised)

## Problem 4: Error analysis

Your last task in this assignment is to do a qualitative error analysis of your information extraction system. You can do this either by writing code or by manual work (inspecting the data file), or mix the two strategies. You can also use the visualisation tools provided by spaCy. For example, the following code cell visualises the output of the named entity recogniser for the given input sentence:

In [ ]:
from spacy import displacy

sentence = u'Slavkov will lose his position as head of the Bulgarian National Olympic Committee.'

displacy.render(nlp(sentence), style='ent', jupyter=True)

In any case, you should enter your pairs in the provided text boxes. Use the triple format shown above, where for each pair you also specify the sentence id (line number in the data file) from which the instance was extracted.

### Recall-related errors (false negatives)

By tuning the `normalise` function, you can deal with some of the recall-related mistakes that your system makes. Other recall-related errors cannot be fixed in this way. To illustrate this, find at least 5&nbsp;entity pairs in the gold standard that your system still does not identify correctly, and enter them into the text box below. For each example, provide a brief explanation of what goes wrong. Try to find examples that illustrate different types of errors.

*Enter your explanations here*

### Precision-related errors (false positives)

Next, provide at least 5 entity pairs that represent false positives of your system. Explain what goes wrong.

*Enter your explanations here*

### Incompleteness of the gold standard

You may have noticed that some of your system&rsquo;s false positives are actually &lsquo;correct&rsquo;. This can happen because, while each entity pair in the gold standard has been manually checked for correctness, no check has been made that the gold standard contains all relevant pairs. Find at least 5&nbsp;entity pairs in the data that are valid instances of the &lsquo;is-leader-of&rsquo; relation (according to your subjective judgement) but that are not contained in the gold standard.

Did you find any examples that you did not find when looking for false positives?

This is the end of the assignment.